In [91]:
from scipy.io import loadmat
import numpy as np

data = loadmat('dataBenchmark.mat')

u_est = np.squeeze(data['uEst'])
y_est = np.squeeze(data['yEst'])
u_val = np.squeeze(data['uVal'])
y_val = np.squeeze(data['yVal'])

In [92]:
scale_parameter = np.abs(np.max(u_est))
u_est /= scale_parameter
u_val /= scale_parameter

In [93]:
R_tab = [15, 16, 17, 18, 19]
kernels_tab = [(100, 60, 10), (100, 60, 20), (80, 40, 10)]
kernels_tab = [(100, 100, 15, 10)]
R_tab = [24, 25, 26]

In [94]:
N = 1024

In [95]:
from volterra import *
from aggregation import *

e_aggr = {}

for R in R_tab:
    for kernels in kernels_tab:
        model_memory_len = np.max(kernels)
        x = u_est[model_memory_len - 1 : N - 1]
        x0 = u_est[0 : model_memory_len - 1]
        y = y_est[model_memory_len - 1 : N - 1]
    
        # aggregation
        m_aggr = VolterraModel(kernels=kernels)
        aggr_parameters = aggregation_for_volterra(m_aggr.dictionary, x, y, x0, R)
        
        # validate
        m_aggr.set_parameters(aggr_parameters)
        y_mod_aggr = m_aggr.evaluate_output(u_val)
        err = 1 / len(u_val) * np.sum((y_mod_aggr - y_val) ** 2)
        
        print("ERR: {0}     R: {1}      kernels: {2}".format(err, R, kernels))

ERR: 1.102033513531323     R: 24      kernels: (100, 100, 15, 10)
ERR: 1.101105961955169     R: 25      kernels: (100, 100, 15, 10)
ERR: 1.135683312539585     R: 26      kernels: (100, 100, 15, 10)
